In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\abhis\\Desktop\\MLProjects\\Movie Recommender'

In [2]:
# Updating the entity (return type of a function) like configbox, and the example below
# same return from config.yaml
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataPreparationConfig:
    root_dir: Path
    source_URL: str
    cache_dir: Path
    movies_data: Path
    links_data: Path
    ratings_data: Path
    # final_data_path: Path
    ratings_data_path: Path
    movies_data_path: Path
    unique_category_path: Path

In [3]:
from MovieRecommender.constants import *
from MovieRecommender.utils.common import read_yaml, create_directories

In [4]:
# update congiuration manager cat configuration.py

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_preparation_config(self) -> DataPreparationConfig:
        data_preparation = self.config.data_preparation
        final_data_dir = os.path.dirname(data_preparation.ratings_data_path)
        create_directories([Path(data_preparation.root_dir),
                            Path(data_preparation.cache_dir),
                            Path(final_data_dir)
                            ])
        folder = self.config.data_ingestion.source_URL.split('/')[-1].split('.zip')[0]
        movies_data = os.path.join(self.config.data_ingestion.unzip_dir, folder, 'movies.csv')
        links_data = os.path.join(self.config.data_ingestion.unzip_dir,folder, 'links.csv')
        ratings_data = os.path.join(self.config.data_ingestion.unzip_dir, folder,'ratings.csv')
        data_preparation_config = DataPreparationConfig(
            root_dir =Path(data_preparation.root_dir),
            source_URL = data_preparation.source_URL,
            cache_dir = Path(data_preparation.cache_dir),
            movies_data = Path(movies_data),
            links_data = Path(links_data),
            ratings_data = Path(ratings_data),
            # final_data_path = Path(data_preparation.final_data_path),
            ratings_data_path = Path(data_preparation.ratings_data_path),
            movies_data_path = Path(data_preparation.movies_data_path), 
            unique_category_path = Path(data_preparation.unique_category_path)
        )
    
        return data_preparation_config

In [5]:
## Update components
import os
import json
import pandas as pd
import numpy as np
import urllib
import urllib.request as request
from MovieRecommender.constants import *
from MovieRecommender.logging import logger
# from MovieRecommender.utils.common import get_size

In [6]:
# source_URL = "https://api.themoviedb.org/3/movie/"
# movie_id = 894

# print(f'{source_URL}{movie_id}?api_key={API_KEY}')

In [7]:
class DataPreparation:
    def __init__(self, config: DataPreparationConfig):
        self.config = config

    def get_df(self, data_path):
        """
        Read a CSV file into a pandas DataFrame.

        Args:
            data_path (str): File path of the CSV file.

        Returns:
            pandas.DataFrame: DataFrame containing the data from the CSV file.
        """
        df = pd.read_csv(data_path)
        return df

    def drop_nan_vals(self, df):
        """
        Drop rows with missing values (NaN) from a DataFrame.

        Args:
            df (pandas.DataFrame): DataFrame to drop missing values from.

        Returns:
            None (modifies df in-place).
        """
        return df.dropna(inplace=True)
    
    def drop_cols(self, df, columns):
        """
        Drop specified columns from a DataFrame.

        Args:
            df (pandas.DataFrame): DataFrame to drop columns from.
            columns (list): List of column names to drop.

        Returns:
            None (modifies df in-place).
        """
        return df.drop(columns = columns, inplace = True)

    def merge_df(self, df1, df2, col_name):
        """
        Merge two DataFrames based on a common column.

        Args:
            df1 (pandas.DataFrame): First DataFrame to merge.
            df2 (pandas.DataFrame): Second DataFrame to merge.
            col_name (str): Name of the common column to merge on.

        Returns:
            pandas.DataFrame: Merged DataFrame.
        """
        merged_df =  df1.merge(df2, on = col_name)
        return merged_df
    def merge_df_with_index(self, df1, df2):
        """
        Merge two DataFrames based on their index.

        Args:
            df1 (pandas.DataFrame): First DataFrame to merge.
            df2 (pandas.DataFrame): Second DataFrame to merge.

        Returns:
            pandas.DataFrame: Merged DataFrame.
        """
        merged_df =  pd.merge(df1, df2, left_index=True, right_index=True)
        return merged_df
    
    def rename_cols(self, df,columns):
        """rename columns

        Args:
            df (pandas): pandas dataframe
            columns (dict): key val with key -> column to be replaced
                            val -> new name
        Returns:
            df: modified df
        """
        return df.rename(columns = columns, inplace=True)
    def reindex_columns(self, df, column_order):
        """
        Reindex the columns of a DataFrame according to the desired order.

        Args:
            df (pandas.DataFrame): The DataFrame to reindex.
            column_order (list): The desired order of columns.

        Returns:
            pandas.DataFrame: The DataFrame with reindexed columns.
        """
        # Reindex the columns
        df = df.reindex(columns=column_order)

        return df
    
    def unique_genres(self,df):
        """
        the total number of unique genres in a DataFrame's 'genres' column.

        Args:
            df (pandas.DataFrame): DataFrame containing a 'genres' column.

        Returns:
            int: Total number of unique genres.

        Usage:
            df = pd.DataFrame({'genres': [['Action', 'Adventure'], ['Comedy', 'Romance'], ['Drama']]})
            total_genres = count_unique_genres(df)
            print(total_genres)  # Output: 4
        """
        genre_column = df['genres']
        unique_genres = set()

        for genres_list in genre_column:
            unique_genres.update(genres_list)

        # total_genres = len(unique_genres)
        return list(unique_genres)

    
    def save_dataframe_to_csv(self,df, path_to_file):
        """
        Save a DataFrame to a specified folder as a CSV file.

        Args:
            df (pandas.DataFrame): The DataFrame to save.
            file_name (str): The name of the CSV file (including the extension).

        Returns:
            None
        """

        
        # Save the DataFrame to CSV
        df.to_csv(path_to_file, index=False)
        
        logger.info(f"DataFrame saved successfully in '{path_to_file}'.")

    def save_dataframe_to_parquet(self,df, path_to_file):
        """
        Save a DataFrame to a specified folder as a parquet file.

        Args:
            df (pandas.DataFrame): The DataFrame to save.
            file_name (str): The name of the CSV file (including the extension).

        Returns:
            None
        """

        
        # Save the DataFrame to CSV
        df.to_parquet(path_to_file)
    def save_dataframe_to_feather(self,df, path_to_file):
        """
        Save a DataFrame to a specified folder as a parquet file.

        Args:
            df (pandas.DataFrame): The DataFrame to save.
            file_name (str): The name of the CSV file (including the extension).

        Returns:
            None
        """

        
        # Save the DataFrame to CSV
        df.to_feather(path_to_file)
    

    def fetch_movie_details_from_api(self,tmdb_id):
        movie_url = f'{self.config.source_URL}{tmdb_id}?api_key={API_KEY}'
        credits_url = f'{self.config.source_URL}{tmdb_id}/credits?api_key={API_KEY}'
        keywords_url = f'{self.config.source_URL}{tmdb_id}/keywords?api_key={API_KEY}'

        movie_data = {}

        try:
            with request.urlopen(movie_url) as response:
                movie_data = json.loads(response.read().decode('utf-8'))

            with request.urlopen(credits_url) as response:
                credits_data = json.loads(response.read().decode('utf-8'))
                crew = credits_data.get('crew', [])
                director = next((member['name'] for member in crew if member.get('job') == 'Director'), np.nan)
                movie_data['director'] = director

            with request.urlopen(keywords_url) as response:
                keywords_data = json.loads(response.read().decode('utf-8'))
                keywords = [keyword['name'] for keyword in keywords_data.get('keywords', [])]
                movie_data['keywords'] = keywords

        except urllib.error.HTTPError as e:
            logger.info(f'Error occurred while requesting movie data {tmdb_id}:', e.code)

        return movie_data

    def fetch_batch_movie_details(self,tmdb_ids):
        movie_data_list = []
        for tmdb_id in tmdb_ids:
            cache_file = Path(os.path.join(self.config.cache_dir, f'{tmdb_id}.json'))

            try:
                with open(cache_file, 'r') as file:
                    movie_data = json.load(file)
            except FileNotFoundError:
                # If the cache file does not exist, fetch the movie details from the API
                movie_data = self.fetch_movie_details_from_api(tmdb_id)
                # Cache the movie details
                with open(cache_file, 'w') as file:
                    json.dump(movie_data, file)
            

            movie_data_list.append(movie_data)
        

        return movie_data_list


    def get_movie_details(self,tmdb_id):
        cache_file = Path(os.path.join(self.config.cache_dir, f'{tmdb_id}.json'))
        try:
            with open(cache_file, 'r') as file:
                movie_data = json.load(file)
        except FileNotFoundError:
            # If the cache file does not exist, fetch the movie details from the API
            movie_data = self.fetch_movie_details_from_api(tmdb_id)
            # Cache the movie details
            with open(cache_file, 'w') as file:
                json.dump(movie_data, file)

        # Extract the desired fields from the movie data
        genres = [genre['name'] for genre in movie_data.get('genres', [])]
        overview = movie_data.get('overview', '')
        popularity = movie_data.get('popularity', 0)
        poster_path = movie_data.get('poster_path', '')
        vote_average = movie_data.get('vote_average', 0)
        vote_count = movie_data.get('vote_count', 0)
        director = movie_data.get('director', np.nan)
        keywords = movie_data.get('keywords', np.nan)

        data = {
            "genres": genres,
            "overview": overview,
            "popularity": popularity,
            "poster_path": poster_path,
            "vote_average": vote_average,
            "vote_count": vote_count,
            "director": director,
            "keywords": keywords
        }

        return data

    def get_batch_movie_details(self,tmdb_ids):

        movie_data_list = self.fetch_batch_movie_details(tmdb_ids)

        result = []
        for movie_data in movie_data_list:
            try:
                tmdb_id = movie_data['id']
            except KeyError:
                tmdb_id = None
            data = self.get_movie_details(tmdb_id)
            result.append(data)
        return result

    ## STEPS 
    def load_data_from_db(self):
        self.movies_df = self.get_df(data_path = self.config.movies_data)
        self.links_df = self.get_df(data_path = self.config.links_data)
        self.ratings_df = self.get_df(data_path = self.config.ratings_data)
        logger.info( f'Data loaded from DB' )

    def preprocess_data(self):
        self.links_movie_df = self.movies_df.merge(self.links_df, on='movieId')
        # drop NAN values
        self.links_movie_df.dropna(inplace=True, axis = 0)
        # change tmdb type from float to int
        self.links_movie_df['tmdbId'] = self.links_movie_df['tmdbId'].astype(int)
        # drop timestamp from ratings_df
        self.ratings_df.drop(columns='timestamp', inplace=True)
        logger.info( f'Data preprocessed' )


    def get_data_from_tmdb(self):
        self.tmdb_ids = self.links_movie_df.tmdbId.values
        self.movie_data = self.get_batch_movie_details(tmdb_ids = self.tmdb_ids)
        logger.info( f'Data loaded from TMDB' )

    def prepare_final_data(self):
        self.tmdb_df = pd.DataFrame(self.movie_data)
        # merge tmdb data and links_movie data
        self.links_movie_df_merged = pd.merge(self.links_movie_df, self.tmdb_df, left_index=True, right_index=True)
        # drop NAN values
        self.links_movie_df_merged.dropna(axis = 0, inplace=True)
        # this will create 2 separate genres, we will drop one of them
        self.links_movie_df_merged.drop(labels='genres_x', axis=1, inplace=True)
        # rename the other genres column
        self.links_movie_df_merged.rename(columns = {'genres_y': 'genres'}, inplace=True)
        # self.final_df = self.links_movie_df_merged.merge(self.ratings_df, on='movieId')
        # # Convert float64 columns to float32
        # float_columns = ['popularity', 'vote_average', 'rating']
        # self.final_df[float_columns] = self.final_df[float_columns].astype('float32')

        # # Convert int64 columns to int32
        # int_columns = ['movieId', 'imdbId', 'tmdbId', 'vote_count', 'userId']
        # self.final_df[int_columns] = self.final_df[int_columns].astype('int32')

        # Define the desired column order
        # column_order = ['movieId', 'imdbId', 'tmdbId', 'title', 
        #                 'genres', 'director', 'overview',
        #         'vote_average', 'vote_count', 'popularity', 
        #         'keywords', 'userId', 'rating', 'poster_path']
        # self.final_df = self.reindex_columns(self.final_df, column_order)
        logger.info( f'Final data prepared' )

    def save_final_data_in_db(self):
        self.save_dataframe_to_csv(self.ratings_df,self.config.ratings_data_path)
        self.save_dataframe_to_csv(self.links_movie_df_merged,self.config.movies_data_path)
        # self.save_dataframe_to_feather(self.final_df, self.config.final_data_path)
        unique_genres_list = self.unique_genres(self.links_movie_df_merged)
        with open(self.config.unique_category_path, 'w') as file:
            json.dump(unique_genres_list, file)

        logger.info(f"Unique genres saved successfully as '{self.config.unique_category_path}'.")
    

    

In [8]:
try:
    config = ConfigurationManager()
    data_preparation_config = config.get_data_preparation_config()
    data_preparation = DataPreparation(config=data_preparation_config)
    data_preparation.load_data_from_db()
    data_preparation.preprocess_data()
    data_preparation.get_data_from_tmdb()
    data_preparation.prepare_final_data()
    data_preparation.save_final_data_in_db()    
except Exception as e:
    raise e

[2023-06-27 18:17:19,610: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-27 18:17:19,614: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-27 18:17:19,616: INFO: common: created directory at: artifacts]
[2023-06-27 18:17:19,619: INFO: common: created directory at: artifacts\data_preparation]
[2023-06-27 18:17:19,621: INFO: common: created directory at: artifacts\data_preparation\cached_data]
[2023-06-27 18:17:19,623: INFO: common: created directory at: artifacts\data_preparation\final_data]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts\\data_ingestion\\ml-latest\\movies.csv'

In [29]:
# final_data = pd.read_feather('artifacts/data_preparation/final_data/final_data.feather')

In [30]:
# final_data.head()

,movieId,imdbId,tmdbId,title,genres,director,overview,vote_average,vote_count,popularity,keywords,userId,rating,poster_path
0,1,114709,862,Toy Story (1995),"[Animation, Adventure, Family, Comedy]",John Lasseter,"Led by Woody, Andy's toys live happily in his ...",8.0,16771,101.402,"[martial arts, jealousy, friendship, bullying,...",4,4.0,/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg
1,1,114709,862,Toy Story (1995),"[Animation, Adventure, Family, Comedy]",John Lasseter,"Led by Woody, Andy's toys live happily in his ...",8.0,16771,101.402,"[martial arts, jealousy, friendship, bullying,...",10,5.0,/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg
2,1,114709,862,Toy Story (1995),"[Animation, Adventure, Family, Comedy]",John Lasseter,"Led by Woody, Andy's toys live happily in his ...",8.0,16771,101.402,"[martial arts, jealousy, friendship, bullying,...",14,4.5,/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg
3,1,114709,862,Toy Story (1995),"[Animation, Adventure, Family, Comedy]",John Lasseter,"Led by Woody, Andy's toys live happily in his ...",8.0,16771,101.402,"[martial arts, jealousy, friendship, bullying,...",15,4.0,/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg
4,1,114709,862,Toy Story (1995),"[Animation, Adventure, Family, Comedy]",John Lasseter,"Led by Woody, Andy's toys live happily in his ...",8.0,16771,101.402,"[martial arts, jealousy, friendship, bullying,...",22,4.0,/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg


In [9]:
source_URL = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

In [13]:
source_URL.split('/')[-1].split('.zip')[0]

'ml-latest-small'